In [1]:
from collections import Counter

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 50)

In [2]:
# Load the CSV file
file_path = "../data/category_triplets_12_12.csv"  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Group by predicted_category and sum the counts
popularity_df = df.groupby("predicted_category", as_index=False)["count"].sum()

# Sort by count in descending order
popularity_df = popularity_df.sort_values(by="count", ascending=False)

# Display the sorted DataFrame
popularity_df

# # Optional: Save the result to a new CSV file
# popularity_df.to_csv('predicted_category_popularity.csv', index=False)


,predicted_category,count
7,Автомобили новые,10
93,Мужчинам,10
175,Уход за телом,9
45,Женщинам,9
49,Запчасти для игровых консолей,7
...,...,...
135,Религиозная атрибутика,1
166,Туризм и отдых на природе,1
158,Техника для общепита,1
160,Товары для гигиены,1


In [3]:
popularity_df.describe()

,count
count,190.000000
mean,3.131579
std,1.778339
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,10.000000


In [4]:
def merge_item_cat2(
    interactions_df: pd.DataFrame,
    items_path: str,
) -> pd.DataFrame:
    """
    Merge items from interactions_df with their item_id from items file.
    """
    # Load data
    items_df = pd.read_csv(items_path, sep=",")

    # Ensure 'cat2' is mapped from items.csv where missing
    interactions_df = interactions_df.merge(
        items_df[["item_id", "cat2"]],
        on="item_id",
        how="left",
        suffixes=("", "_from_items"),
    )

    # Fill missing 'cat2' from items.csv
    if "cat2_from_items" in interactions_df.columns:
        interactions_df["cat2"] = interactions_df["cat2"].combine_first(
            interactions_df["cat2_from_items"]
        )
        interactions_df.drop(columns=["cat2_from_items"], inplace=True, errors="ignore")

    return interactions_df


In [5]:
interactions_df = pd.read_csv(
    "/home/temochka2005/Documents/VScodeProjects/TbankSirius/projectrep/llm-recommender/data/interactions.csv",
    sep=",",
)

In [6]:
interactions_df

,time,user_id,item_id,interaction
0,2024-12-10 12:25:29.446542,621459934,10,1
1,2024-12-10 12:25:31.183529,621459934,6,0
2,2024-12-10 12:25:33.161214,621459934,8,1
3,2024-12-10 12:25:35.448148,621459934,3,0
4,2024-12-10 12:25:37.159775,621459934,10,1
...,...,...,...,...
4797,2024-12-12 16:00:30.466687,856344631,8416,0
4798,2024-12-12 16:00:36.817395,856344631,9633,0
4799,2024-12-12 16:00:40.881110,856344631,12316,1
4800,2024-12-12 16:00:44.190370,856344631,2971,0


In [7]:
interactions_df = merge_item_cat2(
    interactions_df,
    "../data/items.csv",
)

In [8]:
interactions_df

,time,user_id,item_id,interaction,cat2
0,2024-12-10 12:25:29.446542,621459934,10,1,Уход за волосами
1,2024-12-10 12:25:31.183529,621459934,6,0,Уход за волосами
2,2024-12-10 12:25:33.161214,621459934,8,1,Уход за волосами
3,2024-12-10 12:25:35.448148,621459934,3,0,Уход за волосами
4,2024-12-10 12:25:37.159775,621459934,10,1,Уход за волосами
...,...,...,...,...,...
4797,2024-12-12 16:00:30.466687,856344631,8416,0,Инструменты и оборудование
4798,2024-12-12 16:00:36.817395,856344631,9633,0,Сувениры
4799,2024-12-12 16:00:40.881110,856344631,12316,1,Товары для бань и саун
4800,2024-12-12 16:00:44.190370,856344631,2971,0,Женщинам


In [9]:
o = interactions_df.groupby(["cat2"])["interaction"].agg(["count", "mean"])
o1 = interactions_df.groupby(["user_id", "cat2"])["interaction"].agg(["count", "mean"])

cat2_likerate = o.sort_values(by=["count"], ascending=False)

In [10]:
cat2_likerate

,count,mean
cat2,,
Детям,63,0.365079
Женщинам,59,0.406780
Мужчинам,49,0.571429
Охранные системы и видеонаблюдение,39,0.589744
Виниловые пластинки,37,0.540541
...,...,...
Бескаркасная мебель,13,0.538462
Бумажная продукция,13,0.461538
Техника для дома,13,0.769231


In [11]:
cat2_likerate = cat2_likerate.rename(columns={"mean": "like_rate"})

In [12]:
sorted_cat2 = cat2_likerate[cat2_likerate["like_rate"] >= 0.6]
sorted_cat2.sort_values(by=["like_rate", "count"], ascending=False)

,count,like_rate
cat2,,
Электронные сертификаты,18,0.833333
Автомобили новые,27,0.814815
Рули и педали,16,0.812500
Mobile gaming,15,0.800000
Компьютеры и периферия,19,0.789474
Техника для дома,13,0.769231
Личная гигиена,17,0.764706
Уход за телом,18,0.722222
Товары для школы и обучения,18,0.722222


In [13]:
sorted_cat2["normalized_like_rate"] = (
    sorted_cat2["like_rate"] / sorted_cat2["like_rate"].sum()
)
sorted_cat2.sort_values(by=["like_rate", "count"], ascending=False)


/tmp/ipykernel_90313/620331154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_cat2["normalized_like_rate"] = sorted_cat2["like_rate"] / sorted_cat2["like_rate"].sum()


,count,like_rate,normalized_like_rate
cat2,,,
Электронные сертификаты,18,0.833333,0.024539
Автомобили новые,27,0.814815,0.023994
Рули и педали,16,0.812500,0.023926
Mobile gaming,15,0.800000,0.023558
Компьютеры и периферия,19,0.789474,0.023248
Техника для дома,13,0.769231,0.022652
Личная гигиена,17,0.764706,0.022518
Уход за телом,18,0.722222,0.021267
Товары для школы и обучения,18,0.722222,0.021267


In [14]:
sorted_cat2["normalized_like_rate"].sum()

np.float64(1.0000000000000002)

In [15]:
sorted_cat2.describe()

,count,like_rate,normalized_like_rate
count,50.000000,50.000000,50.000000
mean,20.140000,0.679183,0.020000
std,4.223404,0.062925,0.001853
min,11.000000,0.600000,0.017668
25%,17.000000,0.625000,0.018404
50%,20.000000,0.680000,0.020024
75%,23.750000,0.712798,0.020990
max,30.000000,0.833333,0.024539


In [16]:
def generate_triplets(
    interactions_path,
    items_path,
    start_date=None,
    end_date=None,
    min_count=1,
    output_path="category_triplets.csv",
    filtered_categories=None,
):
    """
    Generate triplets of categories with counts, filtered by date range and minimum count.

    :param interactions_path: Path to the interactions CSV file.
    :param items_path: Path to the items CSV file.
    :param start_date: Start date for filtering interactions (inclusive, format: 'YYYY-MM-DD').
    :param end_date: End date for filtering interactions (inclusive, format: 'YYYY-MM-DD').
    :param min_count: Minimum count of triplets to include in the output.
    :param output_path: Path to save the resulting CSV file.
    """
    # Load datasets
    interactions = pd.read_csv(
        interactions_path,
        usecols=["time", "user_id", "item_id", "interaction"],
        sep=",",
    )
    items = pd.read_csv(items_path, usecols=["item_id", "cat2"], sep=",")

    # Map item_id to cat2 category
    items_mapping = items.set_index("item_id")["cat2"]
    interactions["cat2"] = interactions["item_id"].map(items_mapping)
    # Flter by cat2 with like-rate > 0.6
    if filtered_categories is not None:
        interactions = interactions[interactions["cat2"].isin(filtered_categories)]
    # Filter by date range if specified
    if start_date:
        interactions = interactions[interactions["time"] >= start_date]
    if end_date:
        interactions = interactions[interactions["time"] <= end_date]

    # Filter interactions (interaction == 1)
    like_interactions = interactions[interactions["interaction"] == 1].copy()

    # Sort by user_id and time for sequential processing
    like_interactions = like_interactions.sort_values(by=["user_id", "time"])

    # Generate triplets for each user
    triplets = set()
    for user_id, group in like_interactions.groupby("user_id"):
        categories = group["cat2"].dropna().tolist()
        for i in range(len(categories)):
            for j in range(len(categories)):
                for k in range(len(categories)):
                    if len(set([categories[i], categories[j], categories[k]])) == 3:
                        triplets.add(
                            tuple([categories[i], categories[j], categories[k]])
                        )

    # Count occurrences of each triplet
    triplet_counter = Counter(triplets)

    # Create a DataFrame with counts, filtered by min_count
    result_df = pd.DataFrame.from_records(
        (
            (t[0], t[1], t[2], count)
            for t, count in triplet_counter.items()
            if count >= min_count
        ),
        columns=["user_category_1", "user_category_2", "predicted_category", "count"],
    )

    # Filter pairs (user_category_1, user_category_2) with at least 3 distinct predicted_category values
    filtered_df = result_df.groupby(["user_category_1", "user_category_2"]).filter(
        lambda group: group["predicted_category"].nunique() >= 2
    )

    # Sort the filtered DataFrame by count in descending order
    filtered_df = filtered_df.sort_values(by=["user_category_1", "user_category_2"])

    # Save the sorted and filtered triplets with counts to a CSV file
    filtered_df.to_csv(output_path, index=False)

    print(f"Output saved to '{output_path}'.")


# Example usage
generate_triplets(
    interactions_path="../data/interactions.csv",
    items_path="../data/items.csv",
    start_date="2024-12-10",
    end_date="2024-12-13",
    min_count=1,
    output_path="../data/category_triplets_actual_12_12_1111.csv",
    filtered_categories=list(sorted_cat2.index),
)


Output saved to '../data/category_triplets_actual_12_12_1111.csv'.


In [17]:
import pandas as pd

# Load the CSV file
file_path = (
    "../data/category_triplets_actual_12_12_1111.csv"  # Replace with your CSV file path
)
df = pd.read_csv(file_path)

# Create a canonical pair representation (sorted tuple)
df["canonical_pair"] = df.apply(
    lambda row: tuple(sorted([row["user_category_1"], row["user_category_2"]])), axis=1
)

# Identify duplicates based on canonical_pair and predicted_category
df = df.drop_duplicates(subset=["canonical_pair", "predicted_category"], keep="first")

# Drop the auxiliary column
df = df.drop(columns=["canonical_pair"])

# Save the cleaned DataFrame to a new CSV file
# df.to_csv("../data/cleaned_file_category_triplets.csv", index=False)

# Display the result
print(df)

      user_category_1          user_category_2  \
0       Mobile gaming                 Nintendo   
1       Mobile gaming                 Nintendo   
2       Mobile gaming                 Nintendo   
3       Mobile gaming                 Nintendo   
4       Mobile gaming                 Nintendo   
...               ...                      ...   
93641   Уход за телом  Электронные сертификаты   
93642   Уход за телом  Электронные сертификаты   
93643   Уход за телом  Электронные сертификаты   
93644   Уход за телом  Электронные сертификаты   
93645   Уход за телом  Электронные сертификаты   

                           predicted_category  count  
0                Игровые приставки и ноутбуки      1  
1            Посуда и кухонные принадлежности      1  
2                                   Ортопедия      1  
3                   Туризм и отдых на природе      1  
4                       Климатическая техника      1  
...                                       ...    ...  
93641         

In [18]:
df["predicted_category"].value_counts()

predicted_category
Компьютерная и офисная мебель                1122
Аккумуляторы и зарядки                       1115
Туризм и отдых на природе                    1104
Техника для красоты и здоровья               1103
Автомобили новые                             1102
Очки виртуальной реальности                  1086
Климатическая техника                        1080
Демонстрационные доски                       1078
Уход за телом                                1075
Игровые жанры                                1075
Электронные сертификаты                      1074
Личная гигиена                               1073
Игровые наушники                             1072
Витамины и БАДы                              1066
Облачные хранилища                           1048
PlayStation                                  1047
Ретро-консоли                                1044
Загар и защита от солнца                     1032
Игровые приставки                            1030
Блокноты и ежедневники         

In [19]:
predicitons_with_likerate = pd.merge(
    df, sorted_cat2, left_on="predicted_category", right_on="cat2", how="left"
)

In [20]:
sorted_cat2["like_rate_scaled"] = (
    sorted_cat2["like_rate"] - sorted_cat2["like_rate"].min()
) / (sorted_cat2["like_rate"].max() - sorted_cat2["like_rate"].min()) * (1 - 0.2) + 0.2
sorted_cat2

/tmp/ipykernel_90313/2316489414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_cat2["like_rate_scaled"] = (sorted_cat2["like_rate"] - sorted_cat2["like_rate"].min()) / (


,count,like_rate,normalized_like_rate,like_rate_scaled
cat2,,,,
Игровые жанры,30,0.600000,0.017668,0.200000
Сувениры,28,0.607143,0.017879,0.224490
Автомобили новые,27,0.814815,0.023994,0.936508
Мягкая мебель,27,0.629630,0.018541,0.301587
Очки виртуальной реальности,25,0.640000,0.018846,0.337143
Аккумуляторы и зарядки,25,0.680000,0.020024,0.474286
Блокноты и ежедневники,25,0.600000,0.017668,0.200000
PC,25,0.680000,0.020024,0.474286
Аппаратная косметология и массаж,24,0.625000,0.018404,0.285714


In [21]:
sorted_cat2["normalized_like_rate_scaled"] = (
    sorted_cat2["like_rate_scaled"] / sorted_cat2["like_rate_scaled"].sum()
)
sorted_cat2.sort_values(by=["like_rate", "count"], ascending=False)


/tmp/ipykernel_90313/2638944593.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_cat2["normalized_like_rate_scaled"] = (


,count,like_rate,normalized_like_rate,like_rate_scaled,normalized_like_rate_scaled
cat2,,,,,
Электронные сертификаты,18,0.833333,0.024539,1.000000,0.042419
Автомобили новые,27,0.814815,0.023994,0.936508,0.039726
Рули и педали,16,0.812500,0.023926,0.928571,0.039389
Mobile gaming,15,0.800000,0.023558,0.885714,0.037571
Компьютеры и периферия,19,0.789474,0.023248,0.849624,0.036040
Техника для дома,13,0.769231,0.022652,0.780220,0.033096
Личная гигиена,17,0.764706,0.022518,0.764706,0.032438
Уход за телом,18,0.722222,0.021267,0.619048,0.026259
Товары для школы и обучения,18,0.722222,0.021267,0.619048,0.026259


In [22]:
# from tqdm import tqdm

random_choice = np.random.choice(
    sorted_cat2.index, p=sorted_cat2["normalized_like_rate_scaled"], size=75 * 1000
)

category_triplets = pd.read_csv(
    "../data/category_triplets_actual_12_12_1111.csv", sep=","
)


# Подсчёт уникальных элементов и их количества
unique_elements, counts = np.unique(random_choice, return_counts=True)

random_choice_df = pd.DataFrame(
    {"predicted_category": unique_elements, "counts": counts}
)

sampling_df = pd.DataFrame()


for cat, count in zip(unique_elements, counts):
    # Выбираем строки из category_triplets для текущей категории
    matching_rows = category_triplets[category_triplets["predicted_category"] == cat]

    sampled_rows = matching_rows.sample(n=count, replace=True)

    sampling_df = pd.concat([sampling_df, sampled_rows], ignore_index=True)


# Вывод результата
for element, count in zip(unique_elements, counts):
    print(f"Элемент {element} встречается {count} раз(а)")
sampling_df

Элемент Mobile gaming встречается 2833 раз(а)
Элемент Nintendo встречается 1878 раз(а)
Элемент PC встречается 1447 раз(а)
Элемент PlayStation встречается 855 раз(а)
Элемент Xbox встречается 761 раз(а)
Элемент Автомобили новые встречается 2958 раз(а)
Элемент Аккумуляторы и зарядки встречается 1532 раз(а)
Элемент Аксессуары для электроники встречается 619 раз(а)
Элемент Аппаратная косметология и массаж встречается 958 раз(а)
Элемент Бизнес-литература встречается 933 раз(а)
Элемент Блокноты и ежедневники встречается 652 раз(а)
Элемент Видео встречается 1442 раз(а)
Элемент Витамины и БАДы встречается 1681 раз(а)
Элемент Геймпады встречается 1087 раз(а)
Элемент Демонстрационные доски встречается 1721 раз(а)
Элемент Детское питание встречается 1834 раз(а)
Элемент Загар и защита от солнца встречается 1379 раз(а)
Элемент Игровые жанры встречается 677 раз(а)
Элемент Игровые наушники встречается 1772 раз(а)
Элемент Игровые приставки встречается 1516 раз(а)
Элемент Игровые приставки и ноутбуки вс

,user_category_1,user_category_2,predicted_category,count
0,"Ноутбуки, планшеты и электронные книги",Ортопедия,Mobile gaming,1
1,Игровые приставки,Демонстрационные доски,Mobile gaming,1
2,Аппаратная косметология и массаж,Аккумуляторы и зарядки,Mobile gaming,1
3,Личная гигиена,Туризм и отдых на природе,Mobile gaming,1
4,Игровые приставки,Наушники и аудиотехника,Mobile gaming,1
...,...,...,...,...
74995,Компьютеры и периферия,Видео,Электронные сертификаты,1
74996,Транспортные средства,Блокноты и ежедневники,Электронные сертификаты,1
74997,Текстиль,Товары для школы и обучения,Электронные сертификаты,1
74998,Уход за телом,Компьютеры и периферия,Электронные сертификаты,1


In [23]:
sampling_df[sampling_df["predicted_category"] == "Mobile gaming"]

,user_category_1,user_category_2,predicted_category,count
0,"Ноутбуки, планшеты и электронные книги",Ортопедия,Mobile gaming,1
1,Игровые приставки,Демонстрационные доски,Mobile gaming,1
2,Аппаратная косметология и массаж,Аккумуляторы и зарядки,Mobile gaming,1
3,Личная гигиена,Туризм и отдых на природе,Mobile gaming,1
4,Игровые приставки,Наушники и аудиотехника,Mobile gaming,1
...,...,...,...,...
2828,Автомобили новые,Уход за одеждой,Mobile gaming,1
2829,Наушники и аудиотехника,Рули и педали,Mobile gaming,1
2830,Ретро-консоли,PC,Mobile gaming,1
2831,Транспортные средства,Nintendo,Mobile gaming,1


In [24]:
sampling_df.to_csv("../data/sampled_triplets_best.csv", index=False)